In [1]:
#!/usr/bin/env python3.6
# -*- coding: utf-8 -*-

In [2]:
# Imports
import sys
import math
import numpy as np
from matplotlib import animation
from IPython.display import HTML
from matplotlib import pyplot as plt
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'
import mpl_toolkits.mplot3d.axes3d as p3

In [3]:
np.random.seed(20)
np.set_printoptions(threshold=sys.maxsize)

In [4]:
%matplotlib inline

# Data

In [5]:
# Read data
path = '../../../data.csv'
motion = np.genfromtxt(path, delimiter=',', dtype=np.float64)
contacts = np.genfromtxt(path, delimiter=',', usecols=(56, 57), dtype=bool)
print("Motion data has a length of ", len(motion))

Motion data has a length of  17967


In [6]:
print(motion.shape)

(17967, 59)


# Dataset Preparation

In [7]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [8]:
def yaw_from_quaternion(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    x = Q[0]
    y = Q[1]
    z = Q[2]
    w = Q[3]
     
    return np.arctan2(2 * (w*z + x*y), 1 - 2 * (y*y + z*z))

In [9]:
def quaternion_rotation_matrix(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    q0 = Q[3]
    q1 = Q[0]
    q2 = Q[1]
    q3 = Q[2]
     
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
                            
    return rot_matrix

In [14]:
def get_accelerations_footstep_dataset(dataset, footsteps, debug=False):
    idx = 1
    inputs = []
    labels = []
    prev_com_velocity = np.asarray([0, 0, 0])
    
    while idx < len(footsteps):
        if idx > 0:
            # Compute time difference between footsteps
            time_difference = abs(dataset[footsteps[idx], 0] - dataset[footsteps[idx-1], 0])
            
            # Round velocity array
            dataset[footsteps[idx-2], 1:4] = np.round(dataset[footsteps[idx-2], 1:4], 1)
            dataset[footsteps[idx-1], 1:4] = np.round(dataset[footsteps[idx-1], 1:4], 1)
            
            # Skip if data switch
            if dataset[footsteps[idx], -1] != dataset[footsteps[idx-1], -1]:
                idx += 2
                continue
            
            # Continuous data
            if dataset[footsteps[idx], -1] == 0:
                # If accelerating from 0.0 skip the first two steps
                if dataset[footsteps[idx-2], 1] == 0.0:
                    idx += 2
                    continue
                
                # If actual velocity is 0.0 skip
                if dataset[footsteps[idx-1], 1] == 0.0:
                    idx += 1
                    continue
                    
                # If different velocities (non-zero) skip
                if dataset[footsteps[idx-2], 1] != dataset[footsteps[idx-1], 1]:
                    idx += 3
                    continue

            if time_difference < 0.4:
                fl_rr_moving = contacts[footsteps[idx], 0]
                fr_rl_moving = contacts[footsteps[idx], 1]
                
                if fl_rr_moving == fr_rl_moving:
                    idx += 1
                    continue

                # Compute rotation matrix incurred
                R_curr = quaternion_rotation_matrix(dataset[footsteps[idx], 39:43])

                # Retrieve base poses in world frame
                prev_base = dataset[footsteps[idx-1], 16:19]
                curr_base = dataset[footsteps[idx], 16:19]
                
                # Compute world displacement
                world_displacement = curr_base - prev_base
                
                # Compute base displacement
                base_displacement = np.dot(R_curr.T, world_displacement)
                prev_com_displacement = base_displacement
                
                # Compute yaw displacement
                prev_yaw = yaw_from_quaternion(dataset[footsteps[idx-1], 39:43])
                curr_yaw = yaw_from_quaternion(dataset[footsteps[idx], 39:43])
                yaw = curr_yaw - prev_yaw
                
                # Compute velocity
                current_com_velocity = base_displacement/time_difference
                
                inputs.append(dataset[footsteps[idx-2], 1:4].tolist() + # previous cmd
                              dataset[footsteps[idx-1], 1:4].tolist() + # current cmd
                              dataset[footsteps[idx-1], 19:23].tolist() + # current velocity
                              dataset[footsteps[idx-1], 4:16].tolist() + # feet position w.r.t CoM
                              dataset[footsteps[idx-2], 19:23].tolist() + # previous velocity
                              current_com_velocity.tolist() +
                              [fl_rr_moving, fr_rl_moving]) # swiging booleans
                    
                if fl_rr_moving:
                    # FL vectors
                    prev_fl = np.array([[dataset[footsteps[idx-1], 4], dataset[footsteps[idx-1], 5], 0]])
                    curr_fl = np.array([[dataset[footsteps[idx], 4], dataset[footsteps[idx], 5], 0]])

                    # RR vectors
                    prev_rr = np.array([[dataset[footsteps[idx-1], 13], dataset[footsteps[idx-1], 14], 0]])
                    curr_rr = np.array([[dataset[footsteps[idx], 13], dataset[footsteps[idx], 14], 0]])

                    # Feet displacements
                    fl_displacement = curr_fl - prev_fl
                    rr_displacement = curr_rr - prev_rr

                    labels.append([fl_displacement[0,0], fl_displacement[0,1], rr_displacement[0,0], rr_displacement[0,1]])
                else:
                    # FR vectors
                    prev_fr = np.array([[dataset[footsteps[idx-1], 7], dataset[footsteps[idx-1], 8], 0]])
                    curr_fr = np.array([[dataset[footsteps[idx], 7], dataset[footsteps[idx], 8], 0]])

                    # RL vectors
                    prev_rl = np.array([[dataset[footsteps[idx-1], 10], dataset[footsteps[idx-1], 11], 0]])
                    curr_rl = np.array([[dataset[footsteps[idx], 10], dataset[footsteps[idx], 11], 0]])

                    # Feet displacements
                    fr_displacement = curr_fr - prev_fr
                    rl_displacement = curr_rl - prev_rl

                    labels.append([fr_displacement[0,0], fr_displacement[0,1], rl_displacement[0,0], rl_displacement[0,1]])
        
                prev_com_velocity = current_com_velocity
        
        idx += 1
        
    # Convert to numpy array
    inputs_numpy = np.array(inputs)
    labels_numpy = np.array(labels)
    
    return inputs_numpy, labels_numpy

# Stack datasets
X_footsteps_accelerations_absolute, Y_footsteps_accelerations_absolute = get_accelerations_footstep_dataset(motion, np.array([x for x in range(len(motion))]))

# Log
print(X_footsteps_accelerations_absolute.shape)
print(Y_footsteps_accelerations_absolute.shape)

(17248, 31)
(17248, 4)


In [15]:
%store X_footsteps_accelerations_absolute
%store Y_footsteps_accelerations_absolute

Stored 'X_footsteps_accelerations_absolute' (ndarray)
Stored 'Y_footsteps_accelerations_absolute' (ndarray)


In [16]:
Y_footsteps_accelerations_absolute[1]

array([ 0.02931136, -0.00638734,  0.02536489,  0.00131244])

In [17]:
X_footsteps_accelerations_absolute[0, :]

array([ 0.1       ,  0.        ,  0.        ,  0.1       ,  0.        ,
        0.        ,  0.09097511,  0.01569571,  0.03546102, -0.07442539,
        0.25578701,  0.13131595, -0.38037282,  0.22266933, -0.12780198,
       -0.381951  , -0.27515733,  0.12599015, -0.38960645, -0.2416217 ,
       -0.12631081, -0.38173601,  0.0753413 , -0.03407997, -0.01022839,
        0.09069058,  0.09356812, -0.00206157, -0.01183524,  0.        ,
        1.        ])

In [28]:
dic = dict()
for x in range(len(X_footsteps_accelerations_absolute)):
    key = str(round(X_footsteps_accelerations_absolute[x, 0], 1)) + str(round(X_footsteps_accelerations_absolute[x, 3], 1))
    if key in dic:
        dic[key][0].append(Y_footsteps_accelerations_absolute[x, 0])
        dic[key][1].append(X_footsteps_accelerations_absolute[x, 6])
        dic[key][2].append((X_footsteps_accelerations_absolute[x, 10],
                            X_footsteps_accelerations_absolute[x, 13]))
        dic[key][3].append(X_footsteps_accelerations_absolute[x, -5:-4])
    else:
        dic[key] = [[Y_footsteps_accelerations_absolute[x, 0]], 
                    [X_footsteps_accelerations_absolute[x, 6]],
                    [(X_footsteps_accelerations_absolute[x, 10],
                      X_footsteps_accelerations_absolute[x, 13])],
                    [X_footsteps_accelerations_absolute[x, -5:-4]]]

In [32]:
for key in dic.keys():
    print(f"{key[:3]} -> {key[3:]}. Mean: {np.round(np.mean(dic[key][0]), 4)}. Std: {np.round(np.std(dic[key][0]), 3)}.")

0.1 -> 0.1. Mean: 0.0187. Std: 0.016.
0.2 -> 0.2. Mean: 0.0501. Std: 0.016.
0.3 -> 0.3. Mean: 0.0834. Std: 0.017.
0.4 -> 0.4. Mean: 0.1174. Std: 0.019.
0.5 -> 0.5. Mean: 0.153. Std: 0.02.
0.6 -> 0.6. Mean: 0.1844. Std: 0.021.
0.7 -> 0.7. Mean: 0.2157. Std: 0.023.
0.8 -> 0.8. Mean: 0.2473. Std: 0.024.
0.9 -> 0.9. Mean: 0.2781. Std: 0.026.
0.0 -> 0.0. Mean: -0.0047. Std: 0.028.
0.0 -> 0.1. Mean: 0.0097. Std: 0.006.
0.1 -> 0.0. Mean: 0.0066. Std: 0.018.
0.0 -> 0.2. Mean: 0.0201. Std: 0.01.
0.2 -> 0.0. Mean: 0.0273. Std: 0.018.
0.0 -> 0.3. Mean: 0.026. Std: 0.012.
0.3 -> 0.0. Mean: 0.05. Std: 0.02.
0.0 -> 0.4. Mean: 0.0351. Std: 0.019.
0.4 -> 0.0. Mean: 0.0724. Std: 0.019.
0.0 -> 0.5. Mean: 0.0481. Std: 0.023.
0.5 -> 0.0. Mean: 0.0854. Std: 0.035.
0.0 -> 0.6. Mean: 0.0489. Std: 0.026.
0.6 -> 0.0. Mean: 0.1075. Std: 0.029.
0.0 -> 0.7. Mean: 0.0673. Std: 0.032.
0.7 -> 0.0. Mean: 0.1186. Std: 0.041.
0.0 -> 0.8. Mean: 0.0739. Std: 0.039.
0.8 -> 0.0. Mean: 0.148. Std: 0.042.
0.0 -> 0.9. Mean: 0

In [25]:
for key in dic.keys():
    print(f"Prev: {key[:3]}. Actual: {key[3:]}. \n\nDisp: {dic[key]}\n\n.")

Prev: 0.1. Actual: 0.1. 

Disp: [[0.028218790888786316, 0.03515556454658508, 0.02931135892868042, 0.029577597975730896, 0.0324135422706604, 0.0302000492811203, 0.027674898505210876, 0.02716253697872162, 0.02734304964542389, 0.030451297760009766, 0.031034111976623535, 0.023477330803871155, 0.02778378129005432, 0.027541443705558777, 0.03537683188915253, 0.033214688301086426, 0.02937154471874237, 0.02634674310684204, 0.03095404803752899, 0.032969728112220764, 0.028613656759262085, 0.025776207447052002, 0.031421542167663574, 0.03282007575035095, 0.026712268590927124, 0.022576481103897095, 0.026467621326446533, 0.027758091688156128, 0.02947072684764862, 0.0339067280292511, 0.035528942942619324, 0.026207923889160156, 0.027156978845596313, 0.030616283416748047, 0.029754608869552612, 0.027946874499320984, 0.03411939740180969, 0.03495851159095764, 0.028107210993766785, 0.027839839458465576, 0.03053775429725647, 0.03083677589893341, 0.02870088815689087, 0.022518575191497803, 0.03081345558166504,

In [36]:
for x in range(len(X_footsteps_accelerations_absolute)):
    print(f"{X_footsteps_accelerations_absolute[x, 0]} -> {X_footsteps_accelerations_absolute[x, 3]}. Step: {np.round(Y_footsteps_accelerations_absolute[x, 0], 2)}. CoM: {np.round(X_footsteps_accelerations_absolute[x, -5:-4], 2)}. FL: {np.round(X_footsteps_accelerations_absolute[x, 10], 3)}. FR: {np.round(X_footsteps_accelerations_absolute[x, 13], 3)}. RL: {np.round(X_footsteps_accelerations_absolute[x, 16], 3)}. RR: {np.round(X_footsteps_accelerations_absolute[x, 19], 3)}")

0.1 -> 0.1. Step: 0.04. CoM: [0.09]. FL: 0.256. FR: 0.223. RL: -0.275. RR: -0.242
0.1 -> 0.1. Step: 0.03. CoM: [0.1]. FL: 0.228. FR: 0.258. RL: -0.241. RR: -0.269
0.1 -> 0.1. Step: 0.03. CoM: [0.09]. FL: 0.257. FR: 0.227. RL: -0.27. RR: -0.243
0.1 -> 0.1. Step: 0.03. CoM: [0.11]. FL: 0.226. FR: 0.257. RL: -0.242. RR: -0.273
0.1 -> 0.1. Step: 0.03. CoM: [0.09]. FL: 0.259. FR: 0.225. RL: -0.272. RR: -0.239
0.1 -> 0.1. Step: 0.03. CoM: [0.1]. FL: 0.23. FR: 0.253. RL: -0.247. RR: -0.268
0.1 -> 0.1. Step: 0.03. CoM: [0.1]. FL: 0.257. FR: 0.228. RL: -0.271. RR: -0.242
0.1 -> 0.1. Step: 0.03. CoM: [0.09]. FL: 0.227. FR: 0.255. RL: -0.242. RR: -0.271
0.1 -> 0.1. Step: 0.03. CoM: [0.1]. FL: 0.254. FR: 0.228. RL: -0.267. RR: -0.245
0.1 -> 0.1. Step: 0.03. CoM: [0.1]. FL: 0.224. FR: 0.259. RL: -0.241. RR: -0.275
0.1 -> 0.1. Step: 0.02. CoM: [0.08]. FL: 0.255. FR: 0.229. RL: -0.271. RR: -0.243
0.1 -> 0.1. Step: 0.03. CoM: [0.08]. FL: 0.229. FR: 0.253. RL: -0.245. RR: -0.269
0.1 -> 0.1. Step: 0.03.